# E2E Baseline Model

In [18]:
import numpy as np
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.backend import placeholder
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (Concatenate, Conv2D, Dense, Dropout,
                                     Flatten, MaxPooling2D, Reshape,
                                     UpSampling2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, to_categorical

from helpers import can_be_concatenated


In [19]:
input_shape = [160,160,8]

## Numpy basics

In [20]:
x = np.arange(40).reshape(2,4,5)
x

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]]])

In [21]:
x = np.arange(40).reshape(2,2,2,5)
x

array([[[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]]],


       [[[20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]],

        [[30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39]]]])

In [22]:
x = np.arange(20).reshape(2,10)
x

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

In [23]:
y = np.arange(10)
z = np.arange(10,20)
np.stack((y,z))

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])

## Ways to reshape layers

For concatenation the first two array dimensions `[x,x,y]` must be the same in both arrays. The goal is here to resize `branch2` from `[1,1,y]` to `[2,2,y]` to match the shape of `branch1`.

In [45]:
branch1 = placeholder((None, 2,2,512))
branch1.shape

TensorShape([None, 2, 2, 512])

In [46]:
branch2 = placeholder((None, 1,1,256))
branch2.shape

TensorShape([None, 1, 1, 256])

### Using Reshape
Just like with numpy arrays, the sum of the elements in the array must stay the same. This may either be calculated explicitly (`256/(2*2)=64`) or implicitly (`[2,2,-1]`).

In [48]:
Reshape((2,2,64))(branch2).shape

TensorShape([None, 2, 2, 64])

### Using Upsampling

In [50]:
UpSampling2D(size=(2,2))(branch2).shape

TensorShape([None, 2, 2, 256])

## Padding

In [69]:
input_shape = [160,160,8]
inputs = Input(input_shape)

In [70]:
# 32,3,1: filters, kernel_size, strides
conv1 = Conv2D(32, 3, 1, activation='relu')(inputs)
conv1.shape

TensorShape([None, 158, 158, 32])

In [71]:
pool1 = MaxPooling2D(2)(conv1)
pool1.shape

TensorShape([None, 79, 79, 32])

In [72]:
# strides: 1 -> 160,160,32
# strides: 2 ->  80, 80,32
conv1 = Conv2D(32, 3, 1, padding='same',activation='relu')(inputs)
conv1.shape

TensorShape([None, 160, 160, 32])

In [73]:
# Without padding
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 1, activation='relu')(inputs)
conv1 = Conv2D(32, 3, 1, activation='relu')(conv1)
pool1 = MaxPooling2D(2)(conv1)
pool1.shape

TensorShape([None, 78, 78, 32])

In [74]:
# With padding
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(2)(conv1)
pool1.shape

TensorShape([None, 9, 9, 32])

## [Concatenate](https://keras.io/api/layers/merging_layers/concatenate/)
Replaces and splits old `concatenate` function in two separate steps
``` python
up6 = concat([UpSampling2D(size=(2, 2))(conv5), conv4], axis=0)
```

### Concatenate with numpy arrays

In [24]:
x = np.arange(20).reshape(2,2,5)
x

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9]],

       [[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]]])

In [25]:
y = np.arange(20,30).reshape(2,1,5)
y

array([[[20, 21, 22, 23, 24]],

       [[25, 26, 27, 28, 29]]])

In [27]:
# x:  [2,2,5]
# y:  [2,1,5]
# c1: [2,3,5]
c1 = Concatenate(axis=1)([x,y])
c1

<tf.Tensor: shape=(2, 3, 5), dtype=int64, numpy=
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [20, 21, 22, 23, 24]],

       [[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19],
        [25, 26, 27, 28, 29]]])>

In [28]:
# x:  [2,2,5]
# y:  [2,2,3]
# c1: [2,2,8]
y = np.arange(12).reshape(2,2,3)
Concatenate()([x,y])

<tf.Tensor: shape=(2, 2, 8), dtype=int64, numpy=
array([[[ 0,  1,  2,  3,  4,  0,  1,  2],
        [ 5,  6,  7,  8,  9,  3,  4,  5]],

       [[10, 11, 12, 13, 14,  6,  7,  8],
        [15, 16, 17, 18, 19,  9, 10, 11]]])>

### Concatenate with Layers

Dense reshapes lenght of 'row':
``` python
Dense(4)([2,5]) -> [2,4]
Dense(4)([2,2,5]) -> [2,2,4]
```

In [29]:
l1 = Dense(4)(np.arange(10).reshape(2,5))
l1

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[ 2.0660434 ,  0.93344647,  1.5203841 , -1.6713781 ],
       [ 9.013583  ,  5.944206  ,  3.66602   , -5.731333  ]],
      dtype=float32)>

In [30]:
l1 = Dense(4)(np.arange(12).reshape(2,1,6))
l1

<tf.Tensor: shape=(2, 1, 4), dtype=float32, numpy=
array([[[ 4.508412 ,  0.98213  , -1.038495 ,  3.0725205]],

       [[ 4.64798  , -2.4304776, -7.3548145,  9.16238  ]]], dtype=float32)>

In [31]:
l2 = Dense(4)(np.arange(12).reshape(2,2,3))
l2

<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[  2.428606 ,  -0.8935258,  -1.4349568,  -1.4575117],
        [  5.1187563,  -4.587531 ,  -3.8905606,  -5.498067 ]],

       [[  7.8089066,  -8.281536 ,  -6.3461637,  -9.538623 ],
        [ 10.499056 , -11.975542 ,  -8.801768 , -13.579178 ]]],
      dtype=float32)>

In [15]:
l2 = Dense(4)(np.arange(10,20).reshape(2,5))
l2

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-3.87296   , -3.4430141 , -5.340579  ,  0.7713928 ],
       [-5.935388  , -4.8587465 , -8.091616  , -0.01273584]],
      dtype=float32)>

In [32]:
# l1: [2,4]     l1: [2,1,4]
# l2: [2,4]     l2: [2,2,4]
# c1: [2,8]     c2: [2,3,4]
c1 = Concatenate(axis=1)([l1,l2])
c1.shape

TensorShape([2, 3, 4])

In [33]:

l1 = Dense(4)(np.arange(16).reshape(2,2,4))
l2 = Dense(6)(np.arange(16).reshape(2,2,4))
# l1: [2,2,4]
# l2: [2,2,6]
# c2: [2,2,10]
c2 = Concatenate()([l1,l2])
c2

<tf.Tensor: shape=(2, 2, 10), dtype=float32, numpy=
array([[[  0.9441469 ,  -1.0348364 ,  -0.71980906,  -2.946177  ,
          -0.57409763,   0.73893136,  -1.7897422 ,   3.3263345 ,
           0.5791789 ,   1.8379655 ],
        [  3.390298  ,  -0.9375906 ,   0.62057656, -10.316733  ,
          -4.0599937 ,  -0.4908061 ,  -2.1736078 ,   7.6372643 ,
          -0.40104008,   5.0435057 ]],

       [[  5.8364487 ,  -0.8403449 ,   1.9609623 , -17.68729   ,
          -7.54589   ,  -1.7205439 ,  -2.5574734 ,  11.948193  ,
          -1.381259  ,   8.249046  ],
        [  8.282599  ,  -0.74310017,   3.3013482 , -25.057846  ,
         -11.031787  ,  -2.9502811 ,  -2.941339  ,  16.259125  ,
          -2.3614779 ,  11.454586  ]]], dtype=float32)>

In [62]:
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv1)
pool1.shape

TensorShape([None, 9, 9, 32])

### Concatenate the real layers

In [67]:
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(2, padding='same')(conv1)
assert conv1.shape.as_list() == [None, 18, 18, 32]

conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(2, padding='same')(conv2)

conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(2, padding='same')(conv3)

conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(conv4)
assert conv4.shape.as_list() == [None, 1, 1, 256]
pool4 = MaxPooling2D(2, padding='same')(conv4)

conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(conv5)
conv5 = UpSampling2D(2)(conv5)
assert conv5.shape.as_list() == [None, 2, 2, 512]

model = Model(inputs,outputs=conv5)
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 160, 160, 8)]     0         
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 54, 54, 32)        2336      
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 3, 3, 64)          18496     
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 1, 1, 64)          36928     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 1, 1, 64)          0   

In [75]:
conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv4)

conv4.shape

TensorShape([None, 1, 1, 256])

In [36]:
conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(conv5)

conv5.shape

TensorShape([None, 1, 1, 512])

In [37]:
smpl_conv5 = UpSampling2D(2)(conv5)
smpl_conv5.shape

TensorShape([None, 2, 2, 512])

In [39]:
rs_conv4 = Reshape((2,2,64))(conv4)
Concatenate()([smpl_conv5, rs_conv4])

<KerasTensor: shape=(None, 2, 2, 576) dtype=float32 (created by layer 'concatenate_8')>

In [72]:
# [None,2,2,256] would work
X = placeholder(shape=[None, 2, 2, 1], name="Placeholder for conv4")
test_conv4 = Dense(256)(X)
Concatenate()([smpl_conv5, test_conv4])

<KerasTensor: shape=(None, 2, 2, 768) dtype=float32 (created by layer 'concatenate_19')>

In [96]:
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(2)(conv1)
pool1.shape

TensorShape([None, 9, 9, 32])

In [95]:
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(2, padding='same')(conv1)
assert conv1.shape.as_list() == [None, 18, 18, 32]

conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(2, padding='same')(conv2)

conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(2, padding='same')(conv3)

conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(conv4)
assert conv4.shape.as_list() == [None, 1, 1, 256]
pool4 = MaxPooling2D(2, padding='same')(conv4)

conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(conv5)
assert conv5.shape.as_list() == [None, 1, 1, 512]
# make sure that first three dimensions are the same ([None,1,1])
assert can_be_concatenated(conv5, conv4)
#conv5 = UpSampling2D(2)(conv5)
#assert conv5.shape.as_list() == [None, 2, 2, 512]

# Here is where changes are necessary
# conv4: [None, 1, 1, 256]
# conv5: [None, 2, 2, 512]
up6 = Concatenate()([conv5, conv4])
conv6 = Conv2D(256, 3, 3, activation='relu', padding='same')(up6)

model = Model(inputs,outputs=conv6)
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           [(None, 160, 160, 8) 0                                            
__________________________________________________________________________________________________
conv2d_275 (Conv2D)             (None, 54, 54, 32)   2336        input_42[0][0]                   
__________________________________________________________________________________________________
conv2d_276 (Conv2D)             (None, 18, 18, 32)   9248        conv2d_275[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_112 (MaxPooling2D (None, 9, 9, 32)     0           conv2d_276[0][0]                 
___________________________________________________________________________________________

In [ ]:
inputs = Input(input_shape)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv1)

conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, 3, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv2)

conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, 3, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv3)

conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, 3, 3, activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv4)

conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, 3, 3, activation='relu', padding='same')(conv5)
# Dimension 0 in both shapes must be equal, but are 2 and 1. Shapes are [2,512] and [1,256]. for '{{node tf.keras.backend.concatenate/concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](Placeholder, Placeholder_1, tf.keras.backend.concatenate/concat/axis)'
# with input shapes: [?,2,2,512], [?,1,1,256], [] and with computed input tensors: input[2] = <1>.
up6 = concat([UpSampling2D(size=(2, 2))(conv5), conv4], axis=0)
# up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=1)
# up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
conv6 = Conv2D(256, 3, 3, activation='relu', padding='same')(up6)
conv6 = Conv2D(256, 3, 3, activation='relu', padding='same')(conv6)

up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=1)
conv7 = Conv2D(128, 3, 3, activation='relu', padding='same')(up7)
conv7 = Conv2D(128, 3, 3, activation='relu', padding='same')(conv7)

up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=1)
conv8 = Conv2D(64, 3, 3, activation='relu', padding='same')(up8)
conv8 = Conv2D(64, 3, 3, activation='relu', padding='same')(conv8)

up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=1)
conv9 = Conv2D(32, 3, 3, activation='relu', padding='same')(up9)
conv9 = Conv2D(32, 3, 3, activation='relu', padding='same')(conv9)

conv10 = Conv2D(N_Cls, 1, 1, activation='sigmoid')(conv9)

model = Model(input=inputs, output=conv10)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
return model

In [8]:
# Does not work because
# s_c5: [None, 2, 2, 512]
# c4:   [None, 1, 1, 256]
# c4 should be [None, 2, 2, 256]
con1 = Concatenate()([smpl_conv5, conv4])
model = Model(inputs,outputs=con1)
#model.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 2, 2, 512), (None, 1, 1, 256)]